# Emotion detection using CNN 
## A) Creating the model

In [10]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import keras
import keras.utils
from keras import utils as np_utils
from keras.utils.np_utils import to_categorical
import h5py

#### 1) Link for loading the database

##### 1.1) For Romain :

In [25]:
emotion_data = pd.read_csv('/Users/romai/Documents/Ecole/Ingé - M1_auto/Q2/Système intelligent/fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


##### 1.2) For Lucas :

In [2]:
emotion_data = pd.read_csv('/Users/Lucas/Documents/Ingenieur_Industriel/Master_1/Q2/Systèmes_intelligents/fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


#### 2) Reading the data

In [26]:
emotion_data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [27]:
emotion_data.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

In [28]:
emotion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


#### 3) Formation of matrices

In [5]:
# Premièrement, création sous forme de List
X_train = []
y_train = []
X_test = []
y_test = []

# Pour chaque ligne de la base de donnée, si il s'agit d'un "Training" les données vont être attribuées
# à X_train et l'émotion correspondante à y_train
# Si il s'agit d'un "Public test", les données vont être attribuées à X_test et l'émotion correspondante
# à y_test
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k,'float32'))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k,'float32'))
        y_test.append(row['emotion'])

In [12]:
# Transformation du list en array et du type des données en float32
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

# Convertion du vecteur de classe (entiers) en une matrice de classe binaire.
y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

#### 4) Normalization of data between 0 and 1

In [13]:
# Transformation des éléments des matrices en un nombre entre 0 et 1 pour faciliter l'apprentissage
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

# Remodulation de X_train et X_test en image de 48x48 pixels
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# Visualisation de la forme des matrices
print('X_train : ',X_train.shape)
print('y_train : ',y_train.shape)
print('X_test  : ',X_test.shape)
print('y_test  : ',y_test.shape)

X_train :  (28709, 48, 48, 1)
y_train :  (28709, 7)
X_test  :  (3589, 48, 48, 1)
y_test  :  (3589, 7)


#### 5) Creating the model

##### 5.1) Designing the CNN

In [39]:
# Creation d'un model lineaire de piles de couches
model = Sequential()

# 1nst convolution layer
model.add(Convolution2D(64,kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Convolution2D(64,kernel_size=(3,3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.5))

# 2nd convolution layer
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(Convolution2D(64, (3,3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.5))

# 3nd convolution layer
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(Convolution2D(128, (3,3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Passage en matrice à 1D
model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

# Probabilité des classes
model.add(Dense(7, activation='softmax'))

# Affichage du sommaire du model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 9, 9, 64)         

##### 5.2) Model training

In [42]:
# Configuration des pertes et métriques du model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [43]:
# Entrainement du model
model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1, validation_data = (X_test, y_test), shuffle=True)

Epoch 1/50
449/449 [==============================] - 355s 788ms/step - loss: 1.8541 - accuracy: 0.2462 - val_loss: 1.8223 - val_accuracy: 0.2494
Epoch 2/50
449/449 [==============================] - 385s 858ms/step - loss: 1.8001 - accuracy: 0.2558 - val_loss: 1.8075 - val_accuracy: 0.2494
Epoch 3/50
449/449 [==============================] - 368s 820ms/step - loss: 1.7893 - accuracy: 0.2532 - val_loss: 1.7999 - val_accuracy: 0.2497
Epoch 4/50
449/449 [==============================] - 365s 814ms/step - loss: 1.7721 - accuracy: 0.2629 - val_loss: 1.7844 - val_accuracy: 0.2505
Epoch 5/50
449/449 [==============================] - 379s 844ms/step - loss: 1.7647 - accuracy: 0.2666 - val_loss: 1.7717 - val_accuracy: 0.2575
Epoch 6/50
449/449 [==============================] - 363s 808ms/step - loss: 1.7538 - accuracy: 0.2784 - val_loss: 1.7453 - val_accuracy: 0.2984
Epoch 7/50
449/449 [==============================] - 382s 850ms/step - loss: 1.7281 - accuracy: 0.2989 - val_loss: 1.7312 -

In [47]:
loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)

113/113 [==============================] - 15s 137ms/step - loss: 1.1932 - accuracy: 0.5433
[1.193182110786438, 0.5433268547058105]


##### 5.3) Save model with h5py

In [48]:
model.save('model_emotion_detection.h5')

## B) Testing model with camera

In [2]:
import os
import cv2
import numpy as np
import keras
from keras.models import model_from_json
from keras.preprocessing import image
import sklearn.externals
import joblib
import h5py
from tkinter import *
import tensorflow as tf
from PIL import ImageTk, Image
import time

#### 1) Loading the model with H5PY & the face detectcion model

In [3]:
model = keras.models.load_model('model_emotion_detection.h5')
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#### 2) Program performing emotion detection

In [5]:
# Creer une fenetre
window = Tk()
 
# Personnaliser la fenetre
window.title("Emotion recognition")
window.geometry("1200x720")
window.config(background = "darkblue")
 
# Premier texte
label_title = Label(window, text = "Present your face in front of the webcam and click on Start button", font = ("Arial", 25), bg = "darkblue", fg = "white")
label_title.pack() #pour afficher le titre
 
# Second texte
label_subtitle = Label(window, text = "Here's your real time emotion : ", font = ("Arial", 20), bg = "darkblue", fg = "white")
label_subtitle.pack(expand = YES) #pour afficher le titre

# Definition de variables qui vont nous aider pour le programme
global activation_sytem
activation_sytem = 0 

global nbr_iteration
nbr_iteration = 0

global del_label
del_label = 0

# Fonction d'activation de la webcam et du code de reconnaissance facial correspondant
def open_webcam():
    cap = cv2.VideoCapture(0)
    # width, height = 800, 600
    # cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    # cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    activation_sytem = 1
    nbr_iteration = 0
    predictions_sum = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
    
    # Variable del_label qui, une fois au dessus de 1, permettra d'éffacer le contenu des Labels de l'interface dans le but de refaire le test de reconnaissance
    global del_label
    if del_label > 0:
        global label_subsubtitle1
        label_subsubtitle1.destroy()
        global label_subsubtitle2
        label_subsubtitle2.destroy()
        global label_subsubtitle3
        label_subsubtitle3.destroy()
        global label_subsubtitle4
        label_subsubtitle4.destroy()
        global label_subsubtitle5
        label_subsubtitle5.destroy()
        global label_subsubtitle6
        label_subsubtitle6.destroy()
        global label_subsubtitle7
        label_subsubtitle7.destroy()
        global label_subtitle8
        label_subtitle8.destroy()
        
    if del_label == 0:
        del_label = 1
    
    # Partie de code lisant le contenu de la webcam et qui attribue une émotion à ce contenu
    while activation_sytem == 1:
        
        # Lecture de la webcam
        ret,image = cap.read()
        # convertion de l'image en niveau de gris
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Detection du visage sur l'image
        faces_detected = face_haar_cascade.detectMultiScale(converted_image,scaleFactor=1.3, minNeighbors=5)

        for (x,y,w,h) in faces_detected:
            
            # Dessin du rectangle autour du visage
            cv2.rectangle(image,(x,y), (x+w,y+h), (255,0,0))
            # Calibration de l'image en 48x48 pixels
            roi_gray = converted_image[y:y+w,x:x+h]
            roi_gray = cv2.resize(roi_gray,(48,48))
            
            # Transformation de l'image en un array
            image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255
            
            # Prédiction par le model
            predictions = model.predict(image_pixels) 
            max_index = np.argmax(predictions[0])
            
            # Attibution de l'émotion (en string) correspondant à la prédiction du model
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            
            #calcul de la somme des probabilités correspondant à chaque émotion
            predictions_sum[0][0] = predictions_sum[0][0] + predictions[0][0]
            predictions_sum[0][1] = predictions_sum[0][1] + predictions[0][1]
            predictions_sum[0][2] = predictions_sum[0][2] + predictions[0][2]
            predictions_sum[0][3] = predictions_sum[0][3] + predictions[0][3]
            predictions_sum[0][4] = predictions_sum[0][4] + predictions[0][4]
            predictions_sum[0][5] = predictions_sum[0][5] + predictions[0][5]
            predictions_sum[0][6] = predictions_sum[0][6] + predictions[0][6]
            
            # Incrémentation de la variable pour connaître le nombre d'itération effectué et ensuite pouvoir faire une moyenne des émotions
            nbr_iteration = nbr_iteration + 1
            
            # Détermination de la police d'écriture
            font = cv2.FONT_HERSHEY_SIMPLEX
            # Emplacement et propriétés du text "Emotion" placé sur l'image
            cv2.putText(image, emotion_prediction, (int(x), int(y)), font, 3, (0, 0, 255), 2, cv2.LINE_4)
            
            #cv.NamedWindow(window, cv.CV_WINDOW_NORMAL)
            #root = Tk()
            #root.geometry("1200x720")
            #root.config(background = "black")
            #Label(root,text="Rom", font=("times new roman",30,"bold"), bg="black", fg="red").pack()
            #f1 = LabelFrame (root, bg="red")
            #f1.pack()
            #L1 = Label (f1, bg="red")
            #L1.pack()
            #img = ImageTk.PhotoImage(Image.fromarray(image))
            #L1['image'] = img
            
            #root.update()
            #cv2.imshow('Demo', image)
            # Create a frame
            #app = Frame(window, bg="white")
            
            
            # Create a label in the frame
            #main = Label(app)
            #lmain.pack(expand = YES)
            
            #img = Image.fromarray(image)
            #imgtk = ImageTk.PhotoImage(image=img)
            #lmain.imgtk = imgtk
            #lmain.configure(image=imgtk)
            #app.pack(expand = YES)
            
           
            
            #image.canvas = Canvas(window, width=200, height=200, bg = "darkblue")
            #canvas.pack(expand = YES)
           
            
            # Affichage de l'image/vidéo
            cv2.imshow('Demo video', image)
    
           
            # Arrêt de la démonstration une fois avoir atteint le nombre de 60 itérations
            if cv2.waitKey(2) & nbr_iteration == 60:
                
                # Arret de l'utilisation de la webcam
                cap.release()
                
                # calcul de la moyenne des prédictions de chaque émotions sur l'entierté de la démonstration
                moyenne_angry = (predictions_sum[0][0]/nbr_iteration)*100
                mo_angry = round(moyenne_angry,1)
                
                moyenne_disgust = (predictions_sum[0][1]/nbr_iteration)*100
                mo_disgust = round(moyenne_disgust,1)
                
                moyenne_fear = (predictions_sum[0][2]/nbr_iteration)*100
                mo_fear = round(moyenne_fear,1)
                
                moyenne_happy = (predictions_sum[0][3]/nbr_iteration)*100
                mo_happy = round(moyenne_happy,1)
                
                moyenne_sad = (predictions_sum[0][4]/nbr_iteration)*100
                mo_sad = round(moyenne_sad,1)
                
                moyenne_surprise = (predictions_sum[0][5]/nbr_iteration)*100
                mo_surprise = round(moyenne_surprise,1)
                
                moyenne_neutral = (predictions_sum[0][6]/nbr_iteration)*100
                mo_neutral = round(moyenne_neutral,1)
                
                #Determination de l'émotion dominante
                tab_moyenne = [[mo_angry, mo_disgust, mo_fear, mo_happy, mo_sad, mo_surprise, mo_neutral]]
                max_moy = np.argmax(tab_moyenne[0])
                dominant = emotion_detection[max_moy]
                
                #Affichage de ces moyennes sur l'interface
                label_subsubtitle1 = Label(window, text = ('angry', mo_angry,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle2 = Label(window, text = ('disgust', mo_disgust,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle3 = Label(window, text = ('fear', mo_fear,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle4 = Label(window, text = ('happy', mo_happy,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle5 = Label(window, text = ('sad', mo_sad,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle6 = Label(window, text = ('surprise', mo_surprise,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle7 = Label(window, text = ('neutral', mo_neutral,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                
                label_subsubtitle1.pack(expand = YES) 
                label_subsubtitle2.pack(expand = YES) 
                label_subsubtitle3.pack(expand = YES)
                label_subsubtitle4.pack(expand = YES)
                label_subsubtitle5.pack(expand = YES)
                label_subsubtitle6.pack(expand = YES)
                label_subsubtitle7.pack(expand = YES)
                
                #Affichage de l'émotion dominante sur l'interface
                label_subtitle8 = Label(window, text = ('Dominant:', dominant) , font = ("Arial", 30), bg = "darkblue", fg = "red")
                label_subtitle8.pack(expand = YES) #pour afficher le titre
                
                # Fermeture de la fenêtre contenant la webcam
                cv2.destroyAllWindows()
                
                # Arrêt de la boucle for
                break
               
# Fonction Stop permettant la fermeture de la fenêtre d'interface
def stop():
    activation_sytem=0
    window.destroy()
        
#  Ajout des boutons Start et Close permettant donc de lancer la démonstration et de fermer la fenêtre d'interface
button_Stop = Button(window, text = "Close", font =("Arial", 20), bg = 'white', fg = 'darkblue', command = stop)
button_Stop.pack(side = BOTTOM, fill = X, expand = False)

button_Start = Button(window, text = "Start", font =("Arial", 20), bg = 'white', fg = 'darkblue', command = open_webcam)
button_Start.pack(side = BOTTOM, fill = X, expand = False)

# Affichage de l'interface
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\romai\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-5-124a141233fc>", line 66, in open_webcam
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\romai\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-5-124a141233fc>", line 66, in open_webcam
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvt

Source :
- https://analyticsindiamag.com/my-first-cnn-project-emotion-detection-using-convolutional-neural-network-with-tpu/

- https://www.youtube.com/watch?v=DtBu1u5aBsc&t=2363s